## Data aggregation

Goal: create 1 dataset with relevant synthesis parameters and measurement outcomes for everything I've made

Columns in output: UUID, volume fractions of everything, fit results processed with Null if version wasn't best

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import json

## Load synthesis files

In [33]:
synthesis_files = ['/home/bgpelkie/Code/silica-np-synthesis/2024_10_14_highlights_replicates/2024_10_14_highlights_replicates_sampletable.csv', '/home/bgpelkie/Code/silica-np-synthesis/2024_10_07_randomSampleV3/Sample_table_2024_10_07_randomsampleV3.csv', '/home/bgpelkie/Code/silica-np-synthesis/2024_10_04_RandomBaseline_v2/Sample_table_2024_10_04_randomsampleV2.csv', '/home/bgpelkie/Code/silica-np-synthesis/2024_10_17_RandomBaselineV4/Sample_table_2024_10_17_RandomBaselineV4.csv'] 

In [34]:
synth_tables = []

for fp in synthesis_files:
    data= pd.read_csv(fp)
    synth_tables.append(data)

synth_table = pd.concat(synth_tables, axis = 0)

In [35]:
synth_table

,Unnamed: 0.1,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well,ethanol_dilute_vol,sample_dilute_vol
0,0.0,0.0,4fd21940-a331-4524-b730-cf76de1b4d8d,50.000000,124.000000,165.000000,1361.000000,0.362682,7.951422,0.009674,A1,NaN,NaN
1,2.0,2.0,8933d4b1-8929-4b58-a491-a0c55df30fed,83.000000,58.000000,175.000000,1384.000000,0.218193,13.216927,0.016085,A2,NaN,NaN
2,5.0,5.0,130093e4-2947-4d37-b73a-ed4c23442ab8,20.000000,64.000000,49.000000,1567.000000,0.903794,3.190814,0.003982,A3,NaN,NaN
3,6.0,6.0,a3f7d49b-cc77-4468-aae0-ad451027ddc0,181.000000,9.000000,228.000000,1283.000000,0.100743,28.625618,0.034357,A4,NaN,NaN
4,8.0,8.0,f69628da-ff56-4f3a-ba2e-816036b87557,100.000000,52.000000,15.000000,1534.000000,0.182971,15.761151,0.019630,A5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,NaN,35.0,49908a7b-72dd-4050-aab5-05b2807f82f8,60.410553,94.258186,21.613657,1523.717603,0.452263,9.564710,0.011908,E8,931.0,769.0
36,NaN,36.0,5dca1867-c2bf-4e16-893b-57ec828b4951,94.733145,141.702552,95.777438,1367.786865,0.288404,14.998954,0.018328,F1,1210.0,490.0
37,NaN,37.0,21ae2275-1852-4a8d-b2dc-6ec0b850491a,83.552676,104.173866,91.429381,1420.844077,0.326996,13.228767,0.016243,F2,1144.0,556.0
38,NaN,38.0,d112b2bd-0447-459e-b875-15fb7e52b51f,113.713415,66.366096,222.160905,1297.759584,0.240266,18.004071,0.021676,F3,1292.0,408.0


In [36]:
synth_table = synth_table.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis = 1)
synth_table = synth_table.drop(['well'], axis = 1)

In [37]:
synth_table = synth_table.set_index('uuid')

## Load fitting file

In [38]:
with open('sasview_fit_batch_2024_10_24.json', 'rt') as f:
    fit_results = json.load(f)
    

## Get volume fractions

In [39]:
def target_vol(row):
    target_vol = row['teos_volume'] + row['ammonia_volume'] + row['water_volume'] + row['ethanol_volume']
    return target_vol
    

In [40]:
synth_table['target_volume'] = synth_table.apply(target_vol, axis=1)

In [41]:
synth_table['teos_vol_frac'] = synth_table['teos_volume']/synth_table['target_volume']
synth_table['ammonia_vol_frac'] = synth_table['ammonia_volume']/synth_table['target_volume']
synth_table['water_vol_frac'] = synth_table['water_volume']/synth_table['target_volume']
synth_table['ethanol_vol_frac'] = synth_table['ethanol_volume']/synth_table['target_volume']

In [42]:
synth_table

,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,target_volume,teos_vol_frac,ammonia_vol_frac,water_vol_frac,ethanol_vol_frac
uuid,,,,,,,,,,,,,,
4fd21940-a331-4524-b730-cf76de1b4d8d,50.000000,124.000000,165.000000,1361.000000,0.362682,7.951422,0.009674,NaN,NaN,1700.0,0.029412,0.072941,0.097059,0.800588
8933d4b1-8929-4b58-a491-a0c55df30fed,83.000000,58.000000,175.000000,1384.000000,0.218193,13.216927,0.016085,NaN,NaN,1700.0,0.048824,0.034118,0.102941,0.814118
130093e4-2947-4d37-b73a-ed4c23442ab8,20.000000,64.000000,49.000000,1567.000000,0.903794,3.190814,0.003982,NaN,NaN,1700.0,0.011765,0.037647,0.028824,0.921765
a3f7d49b-cc77-4468-aae0-ad451027ddc0,181.000000,9.000000,228.000000,1283.000000,0.100743,28.625618,0.034357,NaN,NaN,1701.0,0.106408,0.005291,0.134039,0.754262
f69628da-ff56-4f3a-ba2e-816036b87557,100.000000,52.000000,15.000000,1534.000000,0.182971,15.761151,0.019630,NaN,NaN,1701.0,0.058789,0.030570,0.008818,0.901822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49908a7b-72dd-4050-aab5-05b2807f82f8,60.410553,94.258186,21.613657,1523.717603,0.452263,9.564710,0.011908,931.0,769.0,1700.0,0.035536,0.055446,0.012714,0.896304
5dca1867-c2bf-4e16-893b-57ec828b4951,94.733145,141.702552,95.777438,1367.786865,0.288404,14.998954,0.018328,1210.0,490.0,1700.0,0.055725,0.083354,0.056340,0.804581
21ae2275-1852-4a8d-b2dc-6ec0b850491a,83.552676,104.173866,91.429381,1420.844077,0.326996,13.228767,0.016243,1144.0,556.0,1700.0,0.049149,0.061279,0.053782,0.835791


In [49]:
fail_count = 0

fits = []
for i, row in synth_table.iterrows():
    uuid_val = row.name

    try:
        fit = fit_results[uuid_val]
        fit_status = 'Success'
    except:
        print('failed fit for, uuid_val')
        fail_count +=1
        fit_status = 'Failed'
        fit = {'best_fit': None,
                 'd_nm': None,
                 'pdi': None,
                 'pl_power': None,
                 'd_gprg': None}
    fit['fit_status'] = fit_status
    fit['uuid'] = uuid_val
    fits.append(fit)

failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val
failed fit for, uuid_val


In [50]:
row.name

'69bb4bea-49e0-4142-b3d6-7c4fbe1993f7'

In [51]:
fit_df = pd.DataFrame(fits)
fit_df = fit_df.set_index('uuid')

In [52]:
synth_fit_table = pd.concat([synth_table, fit_df], axis = 1)

In [53]:
synth_fit_table

,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,target_volume,teos_vol_frac,ammonia_vol_frac,water_vol_frac,ethanol_vol_frac,best_fit,d_nm,pdi,pl_power,d_gprg,fit_status
uuid,,,,,,,,,,,,,,,,,,,,
4fd21940-a331-4524-b730-cf76de1b4d8d,50.000000,124.000000,165.000000,1361.000000,0.362682,7.951422,0.009674,NaN,NaN,1700.0,0.029412,0.072941,0.097059,0.800588,None,NaN,NaN,NaN,NaN,Failed
8933d4b1-8929-4b58-a491-a0c55df30fed,83.000000,58.000000,175.000000,1384.000000,0.218193,13.216927,0.016085,NaN,NaN,1700.0,0.048824,0.034118,0.102941,0.814118,None,NaN,NaN,NaN,NaN,Failed
130093e4-2947-4d37-b73a-ed4c23442ab8,20.000000,64.000000,49.000000,1567.000000,0.903794,3.190814,0.003982,NaN,NaN,1700.0,0.011765,0.037647,0.028824,0.921765,sphere,113.992098,0.172859,NaN,NaN,Success
a3f7d49b-cc77-4468-aae0-ad451027ddc0,181.000000,9.000000,228.000000,1283.000000,0.100743,28.625618,0.034357,NaN,NaN,1701.0,0.106408,0.005291,0.134039,0.754262,guinier_porod,NaN,NaN,NaN,99.778936,Success
f69628da-ff56-4f3a-ba2e-816036b87557,100.000000,52.000000,15.000000,1534.000000,0.182971,15.761151,0.019630,NaN,NaN,1701.0,0.058789,0.030570,0.008818,0.901822,guinier_porod,NaN,NaN,NaN,99.778936,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49908a7b-72dd-4050-aab5-05b2807f82f8,60.410553,94.258186,21.613657,1523.717603,0.452263,9.564710,0.011908,931.0,769.0,1700.0,0.035536,0.055446,0.012714,0.896304,None,NaN,NaN,NaN,NaN,Failed
5dca1867-c2bf-4e16-893b-57ec828b4951,94.733145,141.702552,95.777438,1367.786865,0.288404,14.998954,0.018328,1210.0,490.0,1700.0,0.055725,0.083354,0.056340,0.804581,None,NaN,NaN,NaN,NaN,Failed
21ae2275-1852-4a8d-b2dc-6ec0b850491a,83.552676,104.173866,91.429381,1420.844077,0.326996,13.228767,0.016243,1144.0,556.0,1700.0,0.049149,0.061279,0.053782,0.835791,None,NaN,NaN,NaN,NaN,Failed


In [55]:
len(synth_fit_table[synth_fit_table['best_fit'] == 'sphere'])

11